<a href="https://colab.research.google.com/github/AbirAhmed72/Decision-Table-Generator/blob/decision_table_feature/decision_table_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library Imports**

In [307]:
import pandas as pd
import itertools
import re
import string
import pandas as pd
import spacy

from word2number import w2n
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

# **Inputs**

In [308]:
# sentences = [
#     "If the working hours are 48 or less then give normal salary.",
#     "The rate of the salary is 1.25 times if the working hours exceed 48 on normal working days.",
#     "If the working days are holidays or Sundays then the rate of the salary is 2.00 times."
# ]
sentences = [
    "If the order is from DGS&D then the discount is 10%.",
    "The discount is 15% if the order is from agents and the amount is more than Rs 50000.",
    "The discount is 10% if the order is from retailer and the amount is more than Rs 50000.",
    "If the amount is between Rs 20000 to Rs 50000 and the order is from agents then the discount is 12%.",
    "If the amount is between Rs 20000 to Rs 50000 and the order is from retailer then the discount is 8%.",
    "If the amount is less than Rs 20000 and the order is from agents then the discount is 8%.",
    "The discount is 5% if the order is from retailer and the amount is less than Rs 20000.",
    "The discount is 10% if item is furniture.",
]

# **Condition and action stubs generation**

In [309]:
def create_single_conditions(mixed_conditions):
  mixed_conditions=[item.split(' and ') for item in mixed_conditions]
  unique_conditions=[]
  for conditions in mixed_conditions:
    for condition in conditions:
      unique_conditions.append(condition)
  return unique_conditions

def filter_unique(single_conditions):
  unique_conditions = set()
  for condition in single_conditions:
    unique_conditions.add(condition)
  return list(unique_conditions)

def clean_data(dataset):
  dataset=[data.lower() for data in dataset]
  translator = str.maketrans('', '', string.punctuation)
  return [data.translate(translator) for data in dataset]

def cleaning_action_stubs(mixed_actions):
  return filter_unique(clean_data(mixed_actions))

def cleaning_condition_stubs(mixed_conditions):
  single_conditions= create_single_conditions(clean_data(mixed_conditions))
  return filter_unique(single_conditions)

def cleaning_stubs(mixed_conditions,mixed_actions):
  return cleaning_condition_stubs(mixed_conditions),cleaning_action_stubs(mixed_actions)

def get_stubs(sentences):
  condition_stub = []
  action_stub = []
  for sentence in sentences:
      words = sentence.split()
      if words[0].lower() == 'if':
          condition_index = sentence.lower().index('if') + 2
          then_index = sentence.lower().index('then')
          condition_stub.append(sentence[condition_index:then_index].strip())
          action_stub.append(sentence[then_index + 4:].strip())
      else:
          if_index = sentence.lower().index('if')
          condition_stub.append(sentence[if_index + 2:].strip())
          action_stub.append(sentence[:if_index].strip())

  conditions,actions =(cleaning_stubs(condition_stub,action_stub))
  return conditions,actions


# **Rule Generation**

In [310]:
def get_conditions_for_a_rule(mixed_condition,condition_stubs):
  conditions_for_a_rule=[]

  conditions=split_mixed_stub(mixed_condition)
  for condition in conditions:
    condition=condition.strip()
    for index, condition_stub in enumerate(condition_stubs):
      if condition in condition_stub:
        conditions_for_a_rule.append(index)
  return conditions_for_a_rule

def get_actions_for_a_rule(mixed_action,action_stubs):
  actions_for_a_rule=[]

  actions=split_mixed_stub(mixed_action)
  for action in actions:
    action=action.strip()
    for index, action_stub in enumerate(action_stubs):
      if action in action_stub:
        actions_for_a_rule.append(index)
  return actions_for_a_rule

def parse_sentence(sentence,if_then):
  condition_index = sentence.index('if')
  if(if_then):
    then_index = sentence.index('then')
    mixed_condition=sentence[condition_index+2:then_index].strip()
    mixed_action=sentence[then_index + 4:].strip()
  else:
    mixed_condition=sentence[condition_index+2:].strip()
    mixed_action=sentence[:condition_index].strip()
  return mixed_condition,mixed_action

def get_condition_and_action_from_sentence(sentence):
  sentence=''.join(clean_data(sentence))
  words = sentence.split()

  if words[0] == 'if':
    mixed_condition,mixed_action=parse_sentence(sentence,True)
  else:
    mixed_condition,mixed_action=parse_sentence(sentence,False)
  return mixed_condition,mixed_action

def split_mixed_stub(mixed_condition):
  return mixed_condition.split('and')

def create_rules(sentences,condition_stubs,action_stubs):

  rules={}
  rule=1
  for sentence in sentences:

    mixed_condition,mixed_action =  get_condition_and_action_from_sentence(sentence)
    conditions_for_a_rule =   get_conditions_for_a_rule(mixed_condition,condition_stubs)
    actions_for_a_rule =   get_actions_for_a_rule(mixed_action,action_stubs)

    rules[rule]=[conditions_for_a_rule,actions_for_a_rule]
    rule+=1
  return rules


# **Table Generation**

In [311]:
def make_column_headers(rules):
  column_headers = []
  for i in rules:
    column_header="R"+str(i)
    column_headers.append(column_header)
  return column_headers

def make_row_headers(condition_stubs,action_stubs):
  row_headers = []

  for index,condition_stub in enumerate(condition_stubs):
    row_header="C"+str(index+1)+":"+str(condition_stub)
    row_headers.append(row_header)

  for index,action_stub in enumerate(action_stubs):
    row_header="A"+str(index+1)+":"+str(action_stub)
    row_headers.append(row_header)

  return row_headers

def make_table(condition_stubs,action_stubs,rules):
  column_headers=make_column_headers(rules)
  row_headers=make_row_headers(condition_stubs,action_stubs)
  table = pd.DataFrame(columns=column_headers, index=row_headers)
  return table

def initialize_table(table):
  for row in table.index:
      if(row.startswith('C')):
        for col in table.columns:
                table.at[row, col] = 'I'
      else:
        for col in table.columns:
          table.at[row, col] = ' '
  return table

def adjust_dependent_entries(table,condition,condition_stubs,col):
  dependencies=dependency_dictionary[find_variable_name(condition_stubs[condition])]
  for dependency_condition in dependencies:
    for row in table.index:
      if(row.startswith('C') and dependency_condition!=condition+1 and int(row[1])==dependency_condition):
        table.at[row,col] ='F'

def set_condition_stubs_entries(table,rules,condition_stubs):
  for i in range(1,len(table.columns)+1):
    condition_set=(rules[i][0])
    for condition in condition_set:
      for row in table.index:
        if(row.startswith('C') and int(row[1])==condition+1):
          col=table.columns[i-1]
          table.at[row,col] ='T'
          adjust_dependent_entries(table,condition,condition_stubs,col)
  return table

def set_action_stubs_entries(table,rules):
  for i in range(1,len(table.columns)+1):
    action_set=(rules[i][1])
    for action in action_set:
      for row in table.index:
        if(row.startswith('A') and int(row[1])==action+1):
          col=table.columns[i-1]
          table.at[row,col] ='x'
  return table

def find_independent_rows(table,number_of_conditions,rules):
  independent_rows=[]
  for column_name, conditions in table.items():
    number_of_truths=0
    number_of_ignores=0
    for condition in conditions:
      if(condition=='T'): number_of_truths+=1
      elif(condition=='I'): number_of_ignores+=1
    if(number_of_truths==1 and number_of_ignores==number_of_conditions-1):
      independent_rows.append(rules[int(column_name[1])][0][0])
  return independent_rows

def optimize_table(table,number_of_conditions,rules):
  independent_rows=find_independent_rows(table,number_of_conditions,rules)
  for independent_row in independent_rows:
    i=0
    for row_name, row_data in table.iterrows():
      if(i==independent_row):
        for col_index,data in enumerate(row_data):
          if(data!='T'): table.iat[independent_row, col_index]='F'
      i+=1

def generate_table(condition_stubs,action_stubs,rules):
  table=initialize_table(make_table(condition_stubs,action_stubs,rules))
  table=set_action_stubs_entries(table,rules)
  table=set_condition_stubs_entries(table,rules,condition_stubs)
  optimize_table(table,len(condition_stubs),rules)
  return table

def printDetails(condition_stubs,action_stubs,rules,table):
  print("Rules are: ")
  for rule in rules:

    mixed_condition=""
    for condition in rules[rule][0]:
      mixed_condition+=condition_stubs[condition]
      mixed_condition+=','


    mixed_action=""
    for action in rules[rule][1]:
      mixed_action+=action_stubs[action]
      mixed_action+=','
    print(str(rule)+":"+"If "+mixed_condition+"then "+mixed_action)





# **Test Case generation**
## Task-1 (DONE)
We need to parse the condition and action like this:<br>input: the order is from dgsd
<br>output: order=dgsd

<br>input: the order is from agents
<br>output: order=agents

<br>input: the order is from retailer
<br>output: order=retailer

<br>input: the amount is more than rs 50000
<br>output: amount>=50000


<br>input: the amount is between rs 20000 to rs 50000
<br>output: amount>=20000 &&  amount<=50000


<br>input: the amount is less than rs 20000
<br>output: amount<=20000

<br>input: item is furniture
<br>output: item=furniture

<br>input: the discount is 15
<br>output: discount=15

<br>

## Task-2 (DONE)

From there we need to find out the dependency/toggling of the variables and we will optimize our table

##Task-3

Finally we will generate the test cases after we are finished with optimizing our decision table



In [312]:
def prune_determinant(sentences):
  pruned_sentences=[]
  for sentence in sentences:
    doc = nlp(sentence)
    sentence = " ".join([token.text for token in doc if token.pos_ != "DET"])
    pruned_sentences.append(sentence)
  return pruned_sentences

def describe_parts_of_speech(sentence):
  parts_of_speech={}
  doc = nlp(sentence)
  for token in doc:
    parts_of_speech[token.text]=token.dep_
  return parts_of_speech

def noun_list(sentence):
  nouns=[]
  doc = nlp(sentence)
  for token in doc:
    if(token.pos_=='NOUN'):
      nouns.append(token.text)
  return nouns

def find_variable_name(sentence):
  doc = nlp(sentence)
  for token in doc:
    if(token.pos_=='NOUN'):
      return token.text
  return 'No Noun Found'

def find_patterns(sentence):
  matcher = Matcher(nlp.vocab)

  between_pattern = [
      {"LOWER": "between"},
      {"OP": "*"},
      {"LOWER": "to"}
  ]

  more_than_pattern = [
      {"LOWER": "more"},
      {"LOWER": "than"}
  ]
  less_than_pattern = [
      {"LOWER": "less"},
      {"LOWER": "than"}
  ]

  is_are_pattern = [
      {"LOWER": {"IN": ["is", "are"]},
      }
  ]

  matcher.add("is_are", [is_are_pattern])
  matcher.add("between_to", [between_pattern])
  matcher.add("more_than", [more_than_pattern])
  matcher.add("less_than", [less_than_pattern])

  doc = nlp(sentence)
  matches = matcher(doc)
  matcher_names=[]
  if matches:
    for match_id, start, end in matches:
        matched_text = doc[start:end].text
        matcher_names.append(doc.vocab.strings[match_id])


  return (matcher_names)

def find_sign(sentence):
  sign=""

  patterns=find_patterns(sentence)
  for pattern in patterns:
    if(pattern=='is_are'):
      sign+='='
    elif(pattern=='more_than'):
      sign+='>'
    elif(pattern=='less_than'):
      sign+='<'
    elif(pattern=='between_to'):
      temp=[]
      doc= nlp(sentence)
      for token in doc:
        if(token.tag_=='CD'): temp.append(w2n.word_to_num(token.text))
      sign+=str(temp)
  if sign=="": return 'No Sign Found'
  else : return sign

def find_value_of_a_variable(sentence):
  value=[]
  doc=nlp(sentence)
  if 'between' not in sentence:
    for token in doc:
      if (token.dep_=='pobj' or token.dep_=='dobj' or token.dep_=='nummod'or token.dep_=='attr') :
        value.append(token.text)
    return str(value)
  return ''

def convert_to_expression(sentence):
  # patterns = [
  #   (r'is between (\d+) (to|and) (\d+)', r'\1 < amount < \3'),
  #   (r'is between (\d+) and (\d+)', r'<\1 and >\2'),
  #   (r'is between (\d+) to (\d+)', r'<\1 and >\2'),

  #   (r'are between (\d+) and (\d+)', r'<\1 and >\2'),
  #   (r'are between (\d+) to (\d+)', r'<\1 and >\2'),

  #   (r'\bor less\b', '<='),
  #   (r'\bless than or equal to\b', '<='),
  #   (r'\bless than\b', '<'),

  #   (r'\bor more\b', '>='),
  #   (r'\bmore than or equal to\b', '>='),
  #   (r'\bmore than\b', '>'),

  #   (r'\bor greater\b', '>='),
  #   (r'\bgreater than or equal to\b', '>='),
  #   (r'\bgreater than\b', '>'),


  #   (r'\bis not equal to\b', '!='),
  #   (r'\bis equal to\b', '='),
  #   (r'\bis\b', '='),

  #   (r'\bare not equal to\b', '!='),
  #   (r'\bare equal to\b', '='),
  #   (r'\bare\b', '='),
  # ]

  expression=''
  expression+=find_variable_name(sentence)
  expression+=find_sign(sentence)
  expression+=find_value_of_a_variable(sentence)
  return expression

def get_expressions(condition_stubs,action_stubs):
  condition_stubs=prune_determinant(condition_stubs)
  action_stubs=prune_determinant(action_stubs)
  condition_stubs_expressions=[]
  action_stubs_expressions=[]
  for condition in condition_stubs:
    condition_stubs_expressions.append(convert_to_expression(condition))
  for action in action_stubs:
    action_stubs_expressions.append(convert_to_expression(action))
  return  condition_stubs_expressions,action_stubs_expressions

def find_unique_variables(condition_stubs_expressions):
  unique_variable = {}
  for expression in condition_stubs_expressions:
      variable, value = expression.split('=')
      unique_variable[variable] = value
  unique_variables=list(unique_variable.keys())
  return unique_variables

def create_dependency_dictionary(condition_stubs,condition_stubs_expressions):
  unique_variables=find_unique_variables(condition_stubs_expressions)
  dependencies={}
  for variable in unique_variables:
    conditions=[]
    i=1
    for condition in condition_stubs:
      if(variable==find_variable_name(condition)):
        conditions.append(i)
      i+=1
    dependencies[variable]=conditions
  return dependencies


In [313]:
def initialize_test_case_table(table):
  for row in table.index:
    for col in table.columns:
      table.at[row, col] = 'any'
  return table

def make_test_case_table(dependency_dictionary,rules):
  column_headers=list(dependency_dictionary.keys())
  column_headers.append('Expected Output')

  row_headers=[]
  for i in range(1,len(rules)+1):
    row_headers.append(i)
  test_case_table = pd.DataFrame(columns=column_headers, index=row_headers)
  return initialize_test_case_table(test_case_table)

def fill_test_case_inputs(test_case_table,condition_stubs_expressions,rules):
  for index,rule in enumerate(rules.values()):
    row_label=index+1
    for condition in rule[0]:
      splited_expression=condition_stubs_expressions[condition].split('=')
      col_label=(splited_expression[0])
      test_case_table.at[index+1,col_label]=splited_expression[1]
  return test_case_table

def fill_expected_output_section(test_case_table,action_stubs,rules):
  last_column=test_case_table.columns[-1]
  index=1
  for rule in rules.values():
    test_case_table.at[index, last_column] = (action_stubs[rule[1][0]])
    index+=1
  return (test_case_table)

def generate_test_case_table(rules,condition_stubs,action_stubs):
  condition_stubs_expressions,action_stubs_expressions=get_expressions(condition_stubs,action_stubs)
  dependency_dictionary=create_dependency_dictionary(condition_stubs,condition_stubs_expressions)
  test_case_table =make_test_case_table(dependency_dictionary,rules)
  test_case_table=fill_test_case_inputs(test_case_table,condition_stubs_expressions,rules)
  test_case_table=fill_expected_output_section(test_case_table,action_stubs,rules)
  return test_case_table




# **Driver**

In [314]:

condition_stubs,action_stubs=get_stubs(sentences)
rules=create_rules(sentences,condition_stubs,action_stubs)
table=generate_table(condition_stubs,action_stubs,rules)
test_case_table=generate_test_case_table(rules,condition_stubs,action_stubs)

printDetails(condition_stubs,action_stubs,rules,table)
print(table)
print(test_case_table)

Rules are: 
1:If the order is from dgsd,then the discount is 10,
2:If the order is from agents,the amount is more than rs 50000,then the discount is 15,
3:If the order is from retailer,the amount is more than rs 50000,then the discount is 10,
4:If the amount is between rs 20000 to rs 50000,the order is from agents,then the discount is 12,
5:If the amount is between rs 20000 to rs 50000,the order is from retailer,then the discount is 8,
6:If the amount is less than rs 20000,the order is from agents,then the discount is 8,
7:If the order is from retailer,the amount is less than rs 20000,then the discount is 5,
8:If item is furniture,then the discount is 10,
                                              R1 R2 R3 R4 R5 R6 R7 R8
C1:the amount is between rs 20000 to rs 50000  I  F  F  T  T  F  F  I
C2:the order is from agents                    F  T  F  T  F  T  F  I
C3:the order is from retailer                  F  F  T  F  T  F  T  I
C4:the amount is less than rs 20000            I  F  F  